In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np
import pandas as pd
import pickle

import sklearn
from sklearn.model_selection import train_test_split

import os

# Implementation of a sentiment analysis

In [2]:
tf.enable_eager_execution()

In [3]:
dirname = os.getcwd()
dirname = os.path.dirname(dirname)
dataset_path = os.path.join(dirname, 'datasets/')
print(dataset_path)

/media/kandy/hdd/master-thesis/constituency-parsing/datasets/


In [3]:
gloveSet = pd.read_csv(dataset_path+'glove.42B.10d.txt', sep=' ', header=None)
print(gloveSet.shape)
print(gloveSet.head())

gloveWords = gloveSet.iloc[:,0:1]
gloveVectors = gloveSet.iloc[:,1:]

(1783088, 11)
    0        1         2         3         4         5         6       7   \
0    ,  0.18378 -0.121230 -0.119870  0.015227 -0.191210 -0.066074 -2.9876   
1  the -0.20838 -0.149320 -0.017528 -0.028432 -0.060104 -0.264600 -4.1445   
2    .  0.10876  0.002244  0.222130 -0.121020 -0.048959  0.018135 -3.8174   
3  and -0.09611 -0.257880 -0.358600 -0.328870  0.579500 -0.517740 -4.1582   
4   to -0.24837 -0.454610  0.039227 -0.284220 -0.031852  0.263550 -4.6323   

         8         9         10  
0  0.807950  0.067338 -0.131840  
1  0.629320  0.336720 -0.433950  
2 -0.032631 -0.625940 -0.518980  
3 -0.113710 -0.108480 -0.488850  
4  0.013890 -0.539280 -0.084454  


In [4]:
gloveCorpus = set()
gloveWords.iloc[:,0].str.lower().apply(gloveCorpus.add)
print(len(gloveCorpus))

glove_corpus_word_to_int = {}
glove_corpus_int_to_word = {}
for word in gloveCorpus:
    temp = len(glove_corpus_word_to_int)
    glove_corpus_word_to_int[word] = temp
    glove_corpus_int_to_word[temp] = word
#print(imdb_corpus_word_to_int)
#print(imdb_corpus_int_to_word)

UNK = '<unk>'
temp = len(glove_corpus_word_to_int)


outfile = glovePath +'glove_word_corpus.pic'
with open(outfile, 'wb') as pickle_file:
    pickle.dump([gloveCorpus,glove_corpus_word_to_int, glove_corpus_int_to_word], pickle_file)
    
#with open(outfile, 'rb') as pickle_file:    
#   loaded_data1 = pickle.load(pickle_file)

1783088


In [5]:
sentiment_dataset = pd.read_csv(dataset_path+'imdb_labelled.txt', sep='\t', header=None)
sentiment_dataset.rename({0:'comments',1:'sentiment'}, axis='columns', inplace=True) 
print(sentiment_dataset.shape)
print(sentiment_dataset.head())

(748, 2)
                                            comments  sentiment
0  A very, very, very slow-moving, aimless movie ...          0
1  Not sure who was more lost - the flat characte...          0
2  Attempting artiness with black & white and cle...          0
3       Very little music or anything to speak of.            0
4  The best scene in the movie was when Gerardo i...          1


In [6]:
def convert_imdb_corpus_into_int(sentence):
    words = sentence.lower().split()
    words = [word if word in gloveCorpus else UNK for word in words]
    words_to_num = [glove_corpus_word_to_int[word] for word in words]
    return words_to_num

sentiment_dataset_num = sentiment_dataset.copy()
sentiment_dataset_num['comments'] = sentiment_dataset['comments'].apply(convert_imdb_corpus_into_int)
sentiment_dataset_num.head()

,comments,sentiment
0,"[1243201, 1127477, 1127477, 1127477, 1127477, ...",0
1,"[1127477, 1127477, 1127477, 1127477, 1127477, ...",0
2,"[1127477, 305266, 1127477, 1127477, 1127477, 1...",0
3,"[1127477, 1127477, 1127477, 1127477, 1127477, ...",0
4,"[682986, 1127477, 1127477, 893553, 682986, 112...",1


In [7]:
imdb_train, imdb_test = sklearn.model_selection.train_test_split(sentiment_dataset_num, test_size=0.2)

In [8]:
print(imdb_train.head())
print(imdb_train.shape)

print(imdb_test.head())
print(imdb_test.shape)

                                              comments  sentiment
522  [798015, 1127477, 900296, 1127477, 1127477, 11...          0
233  [1127477, 1127477, 1127477, 1243201, 1127477, ...          0
328  [1127477, 1127477, 1127477, 1127477, 1127477, ...          0
272       [1127477, 682986, 1518213, 1127477, 1682641]          0
465  [1127477, 1127477, 1127477, 1127477, 1127477, ...          1
(598, 2)
                                              comments  sentiment
510  [1127477, 1127477, 1264798, 1127477, 1264798, ...          0
575  [682986, 1127477, 893553, 1127477, 1127477, 89...          1
335  [1127477, 1127477, 1127477, 1450383, 1127477, ...          1
642  [1127477, 1127477, 1127477, 1264798, 682986, 1...          0
474  [1127477, 1127477, 1127477, 1243201, 1127477, ...          1
(150, 2)


In [9]:
unkCount = 0
totalCount = 0
def count_UNK(token_list):
    global unkCount, totalCount
    match = glove_corpus_word_to_int[UNK]
    #print(token_list)
    #print(match)
    unkCount += token_list.count(match)
    totalCount += len(token_list)
    
sentiment_dataset_num['comments'].apply(count_UNK)
print(totalCount)
print(unkCount)

14602
11261


In [10]:
embeddings = tfe.Variable(name='embeddings', validate_shape= gloveVectors.shape, 
                          initial_value=gloveVectors.values, 
                          dtype=tf.float32, trainable=False)
w = tfe.Variable(name='w', validate_shape=(gloveVectors.shape[1], 1), 
                 initial_value=0.01 * tf.random_normal(shape=(gloveVectors.shape[1], 1)),
                 dtype=tf.float32)
b = tfe.Variable(name='b', validate_shape=(1, 1),
                 initial_value=0.01 * tf.random_normal(shape=(1, 1)),
                 dtype=tf.float32)

In [29]:
optimizer = tf.train.AdamOptimizer()

epoch = 1000
for i in range(epoch):
    for j in range(imdb_train.shape[0]):
        with tf.GradientTape() as tape:
            words = tf.nn.embedding_lookup(embeddings, imdb_train.iat[j,0])
            #print(words)
            cbow_words_avg = tf.math.reduce_mean(words, axis=0, keepdims=True)
            #print(cbow_words_avg)
            z = tf.matmul(cbow_words_avg, w) + b
            y_predict = tf.sigmoid(z)
            #print(y_predict)
            loss = tf.losses.sigmoid_cross_entropy(tf.constant(imdb_train.iat[j,1], shape=(1,1)), y_predict)
           
            grads = tape.gradient(loss, [w,b])
            #print(grads)
            optimizer.apply_gradients(zip(grads, [w,b]),
                            global_step=tf.train.get_or_create_global_step())
    if(i % 100 == 0):
        print('Epoch ', i+1)
        print(loss.numpy())

Epoch  1
0.7435041
Epoch  2
Epoch  3
Epoch  4
Epoch  5
Epoch  6
Epoch  7
Epoch  8
Epoch  9
Epoch  10
Epoch  11
Epoch  12
Epoch  13
Epoch  14
Epoch  15
Epoch  16
Epoch  17
Epoch  18
Epoch  19
Epoch  20
Epoch  21
Epoch  22
Epoch  23
Epoch  24
Epoch  25
Epoch  26
Epoch  27
Epoch  28
Epoch  29
Epoch  30
Epoch  31
Epoch  32
Epoch  33
Epoch  34
Epoch  35
Epoch  36
Epoch  37
Epoch  38
Epoch  39
Epoch  40
Epoch  41
Epoch  42
Epoch  43
Epoch  44
Epoch  45
Epoch  46
Epoch  47
Epoch  48
Epoch  49
Epoch  50
Epoch  51
Epoch  52
Epoch  53
Epoch  54
Epoch  55
Epoch  56
Epoch  57
Epoch  58
Epoch  59
Epoch  60
Epoch  61
Epoch  62
Epoch  63
Epoch  64
Epoch  65
Epoch  66
Epoch  67
Epoch  68
Epoch  69
Epoch  70
Epoch  71
Epoch  72
Epoch  73
Epoch  74
Epoch  75
Epoch  76
Epoch  77
Epoch  78
Epoch  79
Epoch  80
Epoch  81
Epoch  82
Epoch  83
Epoch  84
Epoch  85
Epoch  86
Epoch  87
Epoch  88
Epoch  89
Epoch  90
Epoch  91
Epoch  92
Epoch  93
Epoch  94
Epoch  95
Epoch  96
Epoch  97
Epoch  98
Epoch  99
Epoch  10

KeyboardInterrupt: 

In [36]:
a = tf.Variable(1.0)
assign = a.assign(2.0)
with tf.control_dependencies([assign]):
  b = a.read_value()
with tf.control_dependencies([b]):
  other_assign = a.assign(3.0)
with tf.control_dependencies([other_assign]):
  # Will print 2.0 because the value was read before other_assign ran. If
  # `a` was a tf.Variable instead, 2.0 or 3.0 could be printed.
  tf.Print(b, [b])

In [37]:
b

<tf.Tensor: id=9309656, shape=(), dtype=float32, numpy=2.0>